In [64]:
import pandas as pd

In [65]:
s = pd.Series('A|B,AB,B,C|D,D|B,A,B,C,A|D'.split(','))

In [66]:
s.str.get_dummies() #same as OneHotEncoding

,A,AB,B,C,D
0,1,0,1,0,0
1,0,1,0,0,0
2,0,0,1,0,0
3,0,0,0,1,1
4,0,0,1,0,1
5,1,0,0,0,0
6,0,0,1,0,0
7,0,0,0,1,0
8,1,0,0,0,1


In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split
X = pd.read_csv("/content/drive/MyDrive/kaggle/home-data-for-ml-course/train.csv",index_col='Id')
X_test_full = pd.read_csv("/content/drive/MyDrive/kaggle/home-data-for-ml-course/test.csv",index_col='Id')
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"] 
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)       

In [69]:
X_train.head(2)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,...,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,1774,1822,1828,0,0,1828,0,0,2,0,3,1,9,1,2007.0,3,774,0,108,0,0,260,0,0,7,2007,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,894,894,894,0,0,894,0,0,1,0,2,1,5,0,1962.0,1,308,0,0,0,0,0,0,0,8,2009,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [70]:
X_train.shape,X_train_full.shape,X_valid.shape,X_valid_full.shape #pd.get_dummies

((1168, 227), (1168, 79), (292, 227), (292, 79))

In [71]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
my_model_1 =XGBRegressor(random_state = 0)
#fit model
my_model_1.fit(X_train,y_train)
predictions_1 = my_model_1.predict(X_valid)

[16:40:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [72]:
r2_score(y_valid,predictions_1)

0.8457574502927248

In [73]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
my_model_2 =XGBRegressor(n_estimators=1,learning_rate=0.05)
#fit model
my_model_2.fit(X_train,y_train)
predictions_2 = my_model_2.predict(X_valid)
r2_score(y_valid,predictions_2) 

[16:40:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


-4.244909409856289

At n_estimators model performed worst its -ve shifting to estimators as 100 and checking

In [74]:
my_model_3 =XGBRegressor(n_estimators=1000,learning_rate=0.05)
#fit model
my_model_3.fit(X_train,y_train)
predictions_3 = my_model_3.predict(X_valid)
r2_score(y_valid,predictions_3) 

[16:40:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.8666223154151476

In [75]:
from sklearn.metrics import mean_absolute_error
print("Models:1",mean_absolute_error(y_valid,predictions_1))
print("Models:2",mean_absolute_error(y_valid,predictions_2))
print("Models:3",mean_absolute_error(y_valid,predictions_3))

Models:1 16803.434690710616
Models:2 172376.23700369222
Models:3 16084.123354559075
